# 플레이어 기여도 시각화 (산점도)

In [4]:
"""
플레이어 기여도 산점도 (Global Mean 대비 모델 예측값 기준)
- 기여도(Ratio) = AI 예측 골드 / 서버 전체 평균 골드
- 1단계: 분(Minute)별 Ratio 계산
- 2단계: 경기(Match)별 Ratio 평균 계산
- 3단계: 선수(Player)별 경기 Ratio의 평균(X축) 및 표준편차(Y축) 계산
- 제외: BOT_DUO (개인 기여도 산출 불가)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import platform
import os
import glob
from datetime import datetime
from pathlib import Path


# ==========================================
# 0. 설정
# ==========================================
def set_korean_font():
    system_name = platform.system()
    if system_name == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows':
        plt.rcParams['font.family'] = 'Malgun Gothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False


# 파일 경로 설정
DATA_PATH = "data/final_features.csv"
MODEL_DIR = "models"
OUTPUT_DIR = Path("output/player_contribution")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# [수정됨] 봇 듀오 제외한 분석 대상 포지션
TARGET_POSITIONS = ['TOP', 'JUNGLE', 'MIDDLE']

# 학습 코드와 동일한 20개 Feature
FEATURE_COLS = [
    'xp', 'minionsKilled', 'jungleMinionsKilled',
    'totalDamageDoneToChampions', 'totalDamageTaken', 'timeEnemySpentControlled',
    'kills', 'deaths', 'assists', 'soloKills', 'turretPlates', 'wardsPlaced', 'objectiveParticipation',
    'xpDiff', 'csDiff', 'damageDiff', 'killsDiff',
    'KDA', 'killParticipation', 'damageShare'
]

MIN_GAMES = 30  # 분석 최소 판수


# ==========================================
# 1. 모델 찾기 및 데이터 로드
# ==========================================
def find_latest_model(position):
    search_pattern = os.path.join(MODEL_DIR, f"xgb_gold_predictor_{position}_*.pkl")
    files = glob.glob(search_pattern)
    if not files:
        return None
    return max(files, key=os.path.getctime)


def process_position_data(position):
    """포지션별 데이터 처리 및 기여도 계산 (3단계 집계)"""
    print(f"\n📂 [{position}] 데이터 분석 시작...")

    # 1. 데이터 로드
    if not os.path.exists(DATA_PATH):
        print(f"❌ 데이터 파일 없음: {DATA_PATH}")
        return pd.DataFrame()

    df = pd.read_csv(DATA_PATH)
    df = df[df['position'] == position].reset_index(drop=True)
    df = df[df['minute'] > 0]  # 0분 제외

    if df.empty:
        print(f"⚠️ {position} 데이터가 없습니다.")
        return pd.DataFrame()

    # 2. 모델 로드
    model_path = find_latest_model(position)
    if not model_path:
        print(f"⚠️ {position} 모델 파일을 찾을 수 없습니다.")
        return pd.DataFrame()

    try:
        model = joblib.load(model_path)
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return pd.DataFrame()

    # 3. [Logic 1] 서버 전체 평균 골드 계산 (분모)
    # 해당 포지션의 분(Minute)별 전체 골드 평균
    global_mean_gold = df.groupby('minute')['totalGold'].mean().to_dict()

    # 4. [Logic 2] AI 예측 골드 계산 (분자)
    print("   AI 예측 수행 중 (퍼포먼스 측정)...")
    X = df[FEATURE_COLS]
    df['predictedGold'] = model.predict(X)

    # 5. [Logic 3] 기여도 Ratio 계산 (매 분마다)
    # Ratio = 내 예측 골드(내 퍼포먼스) / 그 시간대 서버 평균 골드
    df['global_mean'] = df['minute'].map(global_mean_gold)

    # 0으로 나누기 방지
    df['minute_contribution'] = df['predictedGold'] / df['global_mean'].replace(0, 1)

    # 6. [Logic 4] 경기(Match)별 기여도 평균 산출
    # 한 선수가 한 경기에서 보여준 퍼포먼스 평균
    print("   경기별 기여도 집계 중...")
    match_stats = df.groupby(['matchId', 'puuid', 'win'])['minute_contribution'].mean().reset_index()
    match_stats.rename(columns={'minute_contribution': 'match_contribution'}, inplace=True)

    return match_stats


# ==========================================
# 2. 플레이어별 최종 집계
# ==========================================
def aggregate_player_final_stats(match_stats, min_games=10):
    """선수별 최종 통계 (경기 기여도들의 평균 및 변동폭)"""
    print(f"   선수별 최종 통계 산출 (최소 {min_games}판)...")

    player_stats = match_stats.groupby('puuid').agg(
        games=('matchId', 'count'),            # 총 게임 수
        contribution=('match_contribution', 'mean'), # 경기 기여도들의 '평균' (X축)
        volatility=('match_contribution', 'std'),    # 경기 기여도들의 '표준편차' (Y축)
        win_rate=('win', 'mean')               # 승률
    ).reset_index()

    # 최소 게임 수 필터링
    player_stats = player_stats[player_stats['games'] >= min_games]
    player_stats = player_stats.dropna()

    print(f"   ✅ 분석 대상 선수: {len(player_stats):,}명")
    return player_stats


# ==========================================
# 3. 시각화
# ==========================================
def plot_scatter(player_stats, position, save_path):
    set_korean_font()

    if len(player_stats) < 2:
        return

    fig, ax = plt.subplots(figsize=(14, 10))

    x = player_stats['contribution']
    y = player_stats['volatility']
    sizes = player_stats['games']

    # 점 크기
    size_scaled = 50 + (sizes - sizes.min()) / (sizes.max() - sizes.min()) * 550 if sizes.max() > sizes.min() else 100

    # 산점도
    scatter = ax.scatter(x, y, s=size_scaled, c=player_stats['win_rate'],cmap='RdYlBu',alpha=0.9,edgecolors='white',vmin=0.35, vmax=0.65)
    plt.colorbar(scatter, label='승률 (Win Rate)')

    # 기준선 (1.0 = 서버 평균)
    ax.axvline(x=1.0, color='red', linestyle='-', linewidth=1.5, label='서버 평균 (1.0)')
    ax.axhline(y=y.median(), color='gray', linestyle='--', alpha=0.5, label=f'변동폭 중앙값 ({y.median():.3f})')

    # 텍스트 라벨
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()

    ax.text(x_max, y_max, '고체급 + 기복있음\n(무력형/도박)', ha='right', va='top', color='#e74c3c', fontweight='bold')
    ax.text(x_max, y_min, '고체급 + 꾸준함\n(S급 에이스)', ha='right', va='bottom', color='#27ae60', fontweight='bold')
    ax.text(x_min, y_max, '저체급 + 기복있음\n(불안정)', ha='left', va='top', color='#7f8c8d', fontweight='bold')
    ax.text(x_min, y_min, '저체급 + 꾸준함\n(라인전 약함)', ha='left', va='bottom', color='#95a5a6', fontweight='bold')

    ax.set_xlabel('평균 기여도 Ratio (AI Predicted / Server Mean)', fontsize=12)
    ax.set_ylabel('경기력 변동폭 (Standard Deviation)', fontsize=12)
    ax.set_title(f'[{position}] 플레이어 퍼포먼스 분석', fontsize=16, fontweight='bold')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.5)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()
    print(f"   📊 그래프 저장: {save_path}")


def print_top_players(player_stats, position):
    if player_stats.empty: return

    print(f"\n🏆 [{position}] 퍼포먼스 TOP 5 (평균 기여도 기준)")
    # 기여도 높고 변동폭 낮은 순
    top5 = player_stats.sort_values(['contribution', 'volatility'], ascending=[False, True]).head(5)

    output = top5[['puuid', 'games', 'contribution', 'volatility', 'win_rate']].copy()
    output['win_rate'] = (output['win_rate']*100).round(1).astype(str) + '%'
    output['contribution'] = output['contribution'].round(3)
    output['volatility'] = output['volatility'].round(3)

    print(output.to_string(index=False))


# ==========================================
# 4. 메인 실행
# ==========================================
def main():
    for pos in TARGET_POSITIONS:
        # 1. 데이터 처리 (분별 -> 경기별 집계)
        match_stats = process_position_data(pos)
        if match_stats.empty: continue

        # 2. 선수별 최종 집계 (경기별 -> 선수별 평균/표준편차)
        player_stats = aggregate_player_final_stats(match_stats, min_games=MIN_GAMES)
        if player_stats.empty:
            print(f"   ⚠️ 분석할 선수가 부족합니다 (최소 {MIN_GAMES}판).")
            continue

        # 3. 저장 및 시각화
        pos_dir = OUTPUT_DIR / pos
        pos_dir.mkdir(parents=True, exist_ok=True)

        player_stats.to_csv(pos_dir / "player_stats.csv", index=False)
        print_top_players(player_stats, pos)
        plot_scatter(player_stats, pos, pos_dir / "contribution_scatter.png")

    print(f"\n✅ 모든 분석 완료! '{OUTPUT_DIR}' 폴더 확인.")

if __name__ == "__main__":
    main()


📂 [TOP] 데이터 분석 시작...
   AI 예측 수행 중 (퍼포먼스 측정)...
   경기별 기여도 집계 중...
   선수별 최종 통계 산출 (최소 30판)...
   ✅ 분석 대상 선수: 529명

🏆 [TOP] 퍼포먼스 TOP 5 (평균 기여도 기준)
                                                                         puuid  games  contribution  volatility win_rate
L08zbcmCA6M9qyH8nn5BAAvQK6uG09YHkU7lavS1pxV77K1sjFYRHrGx1ua9sJXirBYeEs-6h_QVaw     30         1.137       0.171    63.3%
ZjD7btisgd0Qh22CXJEoB2BnAJanxb-CCXGFiw4YYm1_IGjOov560513tmjQpxau4bqcnAPkl376Zg     59         1.128       0.123    40.7%
S9_mkbTnBtQHQcTJpswhcjWaNnhNO4ozqLbRgHwCW1xehb4hBOkXCWjyfpiVm0p4_9URnFnAQswlNw     36         1.124       0.097    58.3%
bPo6h2MhGdCpHiTjEabLxY5G2VVMyqp8EKX-heM692DzVuuPK6mEmqXuiylngBi7GWMIjOJt9NBBZQ     52         1.123       0.134    51.9%
Q8tuLUQmCbypZIlhhmohcibyLQMb8LpkI4Q_z4m52Ivz9u8ZFJPBrUzxnAL_XEt3zgtKgohw_ZV85A     53         1.121       0.144    56.6%
   📊 그래프 저장: output/player_contribution/TOP/contribution_scatter.png

📂 [JUNGLE] 데이터 분석 시작...
   AI 예측 수행 중 (퍼포먼스 측정)...
   경기

# 플레이어 기여도 시각화 (산점도) - 첼린저 플레이어 표시

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import platform
import os
import glob
from datetime import datetime
from pathlib import Path

# ==========================================
# 0. 설정
# ==========================================
def set_korean_font():
    system_name = platform.system()
    if system_name == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows':
        plt.rcParams['font.family'] = 'Malgun Gothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False

# [설정] 실행 시점의 타임스탬프로 새로운 폴더 생성 (덮어쓰기 방지)
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
BASE_OUTPUT_DIR = Path(f"output/player_contribution/challengers_{current_time}")
BASE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 결과 저장 폴더 생성: {BASE_OUTPUT_DIR}")

# 파일 경로 설정
DATA_PATH = "data/final_features.csv"
TARGET_PUUID_PATH = "data/puuid_challengers_0408.txt"
MODEL_DIR = "models"

TARGET_POSITIONS = ['TOP', 'JUNGLE', 'MIDDLE']

FEATURE_COLS = [
    'xp', 'minionsKilled', 'jungleMinionsKilled',
    'totalDamageDoneToChampions', 'totalDamageTaken', 'timeEnemySpentControlled',
    'kills', 'deaths', 'assists', 'soloKills', 'turretPlates', 'wardsPlaced', 'objectiveParticipation',
    'xpDiff', 'csDiff', 'damageDiff', 'killsDiff',
    'KDA', 'killParticipation', 'damageShare'
]

MIN_GAMES = 30

# ==========================================
# (신규) 타겟 PUUID 로드 함수
# ==========================================
def load_target_puuids(filepath):
    if not os.path.exists(filepath):
        print(f"⚠️ 타겟 파일 없음: {filepath}")
        return []

    with open(filepath, 'r', encoding='utf-8') as f:
        puuids = [line.strip() for line in f if line.strip()]

    print(f"📋 타겟 플레이어(Challengers) {len(puuids)}명 로드 완료.")
    return puuids

# ==========================================
# 1. 모델 찾기 및 데이터 로드
# ==========================================
def find_latest_model(position):
    search_pattern = os.path.join(MODEL_DIR, f"xgb_gold_predictor_{position}_*.pkl")
    files = glob.glob(search_pattern)
    if not files:
        return None
    return max(files, key=os.path.getctime)

def process_position_data(position):
    print(f"\n📂 [{position}] 데이터 분석 시작...")
    if not os.path.exists(DATA_PATH): return pd.DataFrame()

    df = pd.read_csv(DATA_PATH)
    df = df[df['position'] == position].reset_index(drop=True)
    df = df[df['minute'] > 0]

    if df.empty: return pd.DataFrame()

    model_path = find_latest_model(position)
    if not model_path: return pd.DataFrame()

    try:
        model = joblib.load(model_path)
    except: return pd.DataFrame()

    global_mean_gold = df.groupby('minute')['totalGold'].mean().to_dict()

    print("   AI 예측 수행 중...")
    X = df[FEATURE_COLS]
    df['predictedGold'] = model.predict(X)
    df['global_mean'] = df['minute'].map(global_mean_gold)
    df['minute_contribution'] = df['predictedGold'] / df['global_mean'].replace(0, 1)

    print("   경기별 기여도 집계 중...")
    match_stats = df.groupby(['matchId', 'puuid', 'win'])['minute_contribution'].mean().reset_index()
    match_stats.rename(columns={'minute_contribution': 'match_contribution'}, inplace=True)

    return match_stats

# ==========================================
# 2. 플레이어별 최종 집계
# ==========================================
def aggregate_player_final_stats(match_stats, min_games=10):
    print(f"   선수별 최종 통계 산출 (최소 {min_games}판)...")
    player_stats = match_stats.groupby('puuid').agg(
        games=('matchId', 'count'),
        contribution=('match_contribution', 'mean'),
        volatility=('match_contribution', 'std'),
        win_rate=('win', 'mean')
    ).reset_index()

    player_stats = player_stats[player_stats['games'] >= min_games]
    player_stats = player_stats.dropna()
    print(f"   ✅ 분석 대상 선수: {len(player_stats):,}명")
    return player_stats

# ==========================================
# 3. 시각화 (수정됨: 모양 및 테두리 변경)
# ==========================================
def plot_scatter(player_stats, position, save_path, target_puuids=[]):
    set_korean_font()

    if len(player_stats) < 2:
        return

    fig, ax = plt.subplots(figsize=(14, 10))

    # 데이터 분리 (일반 vs 타겟)
    targets = player_stats[player_stats['puuid'].isin(target_puuids)]
    normals = player_stats[~player_stats['puuid'].isin(target_puuids)]

    # 공통: 점 크기 계산 함수
    def get_sizes(data, all_sizes):
        # 전체 데이터의 최소/최대 기준으로 스케일링
        return 50 + (data - all_sizes.min()) / (all_sizes.max() - all_sizes.min()) * 550 if all_sizes.max() > all_sizes.min() else 100

    all_games = player_stats['games']

    # ---------------------------------------------------------
    # 1. 일반 플레이어 그리기 (동그라미 + 흰색 테두리)
    # ---------------------------------------------------------
    if not normals.empty:
        normal_sizes = get_sizes(normals['games'], all_games)
        scatter_normal = ax.scatter(
            normals['contribution'], normals['volatility'],
            s=normal_sizes,
            c=normals['win_rate'],
            cmap='RdYlBu',
            marker='o',             # 동그라미
            edgecolors='white',     # 흰색 테두리 (깔끔함)
            linewidth=0.8,
            alpha=0.7,              # 투명도를 주어 겹쳐도 보이게 함
            vmin=0.35, vmax=0.65,
            label='General Players'
        )
        # 컬러바는 일반 플레이어 기준으로 생성
        plt.colorbar(scatter_normal, label='승률 (Win Rate)')

    # ---------------------------------------------------------
    # 2. 타겟 플레이어 그리기 (다이아몬드 + 검정 테두리)
    # ---------------------------------------------------------
    if not targets.empty:
        print(f"   ✨ 타겟 플레이어 {len(targets)}명 강조 표시 (다이아몬드 + 검정테두리)")
        target_sizes = get_sizes(targets['games'], all_games)

        ax.scatter(
            targets['contribution'], targets['volatility'],
            s=target_sizes,
            c=targets['win_rate'],
            cmap='RdYlBu',
            marker='D',             # 다이아몬드 모양 (구분됨)
            edgecolors='black',     # ★ 검정색 진한 테두리
            linewidth=1.0,          # 테두리 두껍게
            alpha=0.9,              # 불투명 (강조)
            vmin=0.35, vmax=0.65,   # 컬러맵 범위 동일
            label='Challengers (Target)',
            zorder=10               # 맨 위에 그리기
        )

    # 기준선 및 텍스트
    y_median = player_stats['volatility'].median()
    ax.axvline(x=1.0, color='red', linestyle='-', linewidth=1.5, label='서버 평균 (1.0)')
    ax.axhline(y=y_median, color='gray', linestyle='--', alpha=0.5, label=f'변동폭 중앙값 ({y_median:.3f})')

    x_min, x_max = player_stats['contribution'].min(), player_stats['contribution'].max()
    y_min, y_max = player_stats['volatility'].min(), player_stats['volatility'].max()

    ax.text(x_max, y_max, '고체급 + 기복있음\n(무력형/도박)', ha='right', va='top', color='#e74c3c', fontweight='bold')
    ax.text(x_max, y_min, '고체급 + 꾸준함\n(S급 에이스)', ha='right', va='bottom', color='#27ae60', fontweight='bold')
    ax.text(x_min, y_max, '저체급 + 기복있음\n(불안정)', ha='left', va='top', color='#7f8c8d', fontweight='bold')
    ax.text(x_min, y_min, '저체급 + 꾸준함\n(라인전 약함)', ha='left', va='bottom', color='#95a5a6', fontweight='bold')

    ax.set_xlabel('평균 기여도 Ratio (AI Predicted / Server Mean)', fontsize=12)
    ax.set_ylabel('경기력 변동폭 (Standard Deviation)', fontsize=12)
    ax.set_title(f'[{position}] 플레이어 퍼포먼스 분석', fontsize=16, fontweight='bold')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.5)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()
    print(f"   📊 그래프 저장: {save_path}")

def print_top_players(player_stats, position):
    if player_stats.empty: return
    print(f"\n🏆 [{position}] 퍼포먼스 TOP 5")
    top5 = player_stats.sort_values(['contribution', 'volatility'], ascending=[False, True]).head(5)
    output = top5[['puuid', 'games', 'contribution', 'volatility', 'win_rate']].copy()
    output['win_rate'] = (output['win_rate']*100).round(1).astype(str) + '%'
    output['contribution'] = output['contribution'].round(3)
    output['volatility'] = output['volatility'].round(3)
    print(output.to_string(index=False))

# ==========================================
# 4. 메인 실행
# ==========================================
def main():
    target_puuids = load_target_puuids(TARGET_PUUID_PATH)

    for pos in TARGET_POSITIONS:
        match_stats = process_position_data(pos)
        if match_stats.empty: continue

        player_stats = aggregate_player_final_stats(match_stats, min_games=MIN_GAMES)
        if player_stats.empty:
            print(f"   ⚠️ 분석할 선수가 부족합니다.")
            continue

        pos_dir = BASE_OUTPUT_DIR / pos
        pos_dir.mkdir(parents=True, exist_ok=True)

        player_stats.to_csv(pos_dir / "player_stats.csv", index=False)
        print_top_players(player_stats, pos)

        plot_scatter(player_stats, pos, pos_dir / "contribution_scatter.png", target_puuids)

    print(f"\n✅ 모든 분석 완료! 결과 폴더: '{BASE_OUTPUT_DIR}'")

if __name__ == "__main__":
    main()

📁 결과 저장 폴더 생성: output/player_contribution/challengers_only_20251207_223418
📋 타겟 플레이어(Challengers) 300명 로드 완료.

📂 [TOP] 데이터 분석 시작...
   AI 예측 수행 중...
   경기별 기여도 집계 중...
   선수별 최종 통계 산출 (최소 30판)...
   ✅ 분석 대상 선수: 529명

🏆 [TOP] 퍼포먼스 TOP 5
                                                                         puuid  games  contribution  volatility win_rate
L08zbcmCA6M9qyH8nn5BAAvQK6uG09YHkU7lavS1pxV77K1sjFYRHrGx1ua9sJXirBYeEs-6h_QVaw     30         1.137       0.171    63.3%
ZjD7btisgd0Qh22CXJEoB2BnAJanxb-CCXGFiw4YYm1_IGjOov560513tmjQpxau4bqcnAPkl376Zg     59         1.128       0.123    40.7%
S9_mkbTnBtQHQcTJpswhcjWaNnhNO4ozqLbRgHwCW1xehb4hBOkXCWjyfpiVm0p4_9URnFnAQswlNw     36         1.124       0.097    58.3%
bPo6h2MhGdCpHiTjEabLxY5G2VVMyqp8EKX-heM692DzVuuPK6mEmqXuiylngBi7GWMIjOJt9NBBZQ     52         1.123       0.134    51.9%
Q8tuLUQmCbypZIlhhmohcibyLQMb8LpkI4Q_z4m52Ivz9u8ZFJPBrUzxnAL_XEt3zgtKgohw_ZV85A     53         1.121       0.144    56.6%
   ✨ 타겟 플레이어 90명 강조 표시 (다이아몬드 + 검정테두리)

# 챌린저 플레이어 기여도 시각화 결과 예시

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import platform
import os
import glob
from datetime import datetime
from pathlib import Path

# ==========================================
# 0. 설정
# ==========================================
def set_korean_font():
    system_name = platform.system()
    if system_name == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows':
        plt.rcParams['font.family'] = 'Malgun Gothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
BASE_OUTPUT_DIR = Path(f"output/challengers_only_{current_time}")
BASE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 챌린저 기준 분석 결과 폴더: {BASE_OUTPUT_DIR}")

DATA_PATH = "data/final_features.csv"
TARGET_PUUID_PATH = "data/puuid_challengers_0408.txt"
MODEL_DIR = "models"
TARGET_POSITIONS = ['TOP', 'JUNGLE', 'MIDDLE']

FEATURE_COLS = [
    'xp', 'minionsKilled', 'jungleMinionsKilled',
    'totalDamageDoneToChampions', 'totalDamageTaken', 'timeEnemySpentControlled',
    'kills', 'deaths', 'assists', 'soloKills', 'turretPlates', 'wardsPlaced', 'objectiveParticipation',
    'xpDiff', 'csDiff', 'damageDiff', 'killsDiff',
    'KDA', 'killParticipation', 'damageShare'
]

MIN_GAMES = 30

# ==========================================
# (신규) 타겟 PUUID 로드 함수
# ==========================================
def load_target_puuids(filepath):
    if not os.path.exists(filepath):
        print(f"⚠️ 타겟 파일 없음: {filepath}")
        return []
    with open(filepath, 'r', encoding='utf-8') as f:
        puuids = [line.strip() for line in f if line.strip()]
    print(f"📋 타겟 플레이어(Challengers) {len(puuids)}명 로드 완료.")
    return puuids

# ==========================================
# 1. 모델 찾기 및 데이터 로드 (핵심 수정)
# ==========================================
def find_latest_model(position):
    search_pattern = os.path.join(MODEL_DIR, f"xgb_gold_predictor_{position}_*.pkl")
    files = glob.glob(search_pattern)
    if not files: return None
    return max(files, key=os.path.getctime)

def process_position_data(position, target_puuids):
    print(f"\n📂 [{position}] 데이터 분석 시작...")
    if not os.path.exists(DATA_PATH): return pd.DataFrame()

    df = pd.read_csv(DATA_PATH)
    df = df[df['position'] == position].reset_index(drop=True)
    df = df[df['minute'] > 0]

    # 📌 [핵심 변경] 데이터를 로드하자마자 챌린저만 남기고 필터링
    # 이렇게 하면 이후 계산되는 평균(global_mean)이 "챌린저 평균"이 됩니다.
    original_count = len(df)
    df = df[df['puuid'].isin(target_puuids)]
    filtered_count = len(df)

    print(f"   ✂️ 데이터 필터링: {original_count:,}행 → {filtered_count:,}행 (챌린저 데이터만 사용)")

    if df.empty:
        print("   ⚠️ 해당 포지션에 챌린저 데이터가 없습니다.")
        return pd.DataFrame()

    model_path = find_latest_model(position)
    if not model_path: return pd.DataFrame()

    try:
        model = joblib.load(model_path)
    except: return pd.DataFrame()

    # 여기서 구하는 평균은 이제 "챌린저들의 평균 골드"입니다.
    global_mean_gold = df.groupby('minute')['totalGold'].mean().to_dict()

    print("   AI 예측 수행 중...")
    X = df[FEATURE_COLS]
    df['predictedGold'] = model.predict(X)

    # 챌린저 평균 대비 비율 계산
    df['global_mean'] = df['minute'].map(global_mean_gold)
    df['minute_contribution'] = df['predictedGold'] / df['global_mean'].replace(0, 1)

    print("   경기별 기여도 집계 중...")
    match_stats = df.groupby(['matchId', 'puuid', 'win'])['minute_contribution'].mean().reset_index()
    match_stats.rename(columns={'minute_contribution': 'match_contribution'}, inplace=True)

    return match_stats

# ==========================================
# 2. 플레이어별 최종 집계
# ==========================================
def aggregate_player_final_stats(match_stats, min_games=10):
    print(f"   선수별 최종 통계 산출 (최소 {min_games}판)...")
    player_stats = match_stats.groupby('puuid').agg(
        games=('matchId', 'count'),
        contribution=('match_contribution', 'mean'),
        volatility=('match_contribution', 'std'),
        win_rate=('win', 'mean')
    ).reset_index()

    player_stats = player_stats[player_stats['games'] >= min_games]
    player_stats = player_stats.dropna()
    return player_stats

# ==========================================
# 3. 시각화
# ==========================================
def plot_scatter(player_stats, position, save_path):
    set_korean_font()

    if len(player_stats) < 2:
        return

    fig, ax = plt.subplots(figsize=(14, 10))

    # 챌린저 내부에서의 판수 스케일링
    sizes = player_stats['games']
    size_scaled = 50 + (sizes - sizes.min()) / (sizes.max() - sizes.min()) * 550 if sizes.max() > sizes.min() else 100

    scatter = ax.scatter(
        player_stats['contribution'],
        player_stats['volatility'],
        s=size_scaled,
        c=player_stats['win_rate'],
        cmap='RdYlBu',
        marker='D',
        edgecolors='black',
        linewidth=1.0,
        alpha=0.8,
        vmin=0.35, vmax=0.65
    )
    plt.colorbar(scatter, label='승률 (Win Rate)')

    # 기준선 (1.0 = 챌린저 평균)
    ax.axvline(x=1.0, color='red', linestyle='-', linewidth=1.5, label='챌린저 평균 (1.0)')
    y_median = player_stats['volatility'].median()
    ax.axhline(y=y_median, color='gray', linestyle='--', alpha=0.5, label=f'중앙값 ({y_median:.3f})')

    # 축 범위 설정 (데이터에 맞게)
    x_min, x_max = player_stats['contribution'].min(), player_stats['contribution'].max()
    y_min, y_max = player_stats['volatility'].min(), player_stats['volatility'].max()

    x_margin = (x_max - x_min) * 0.1
    y_margin = (y_max - y_min) * 0.1
    ax.set_xlim(x_min - x_margin, x_max + x_margin)
    ax.set_ylim(y_min - y_margin, y_max + y_margin)

    # 4분면 텍스트 (문구 수정: 기준이 챌린저임)
    ax.text(ax.get_xlim()[1], ax.get_ylim()[1], '평균 이상 + 기복있음\n(슈퍼 캐리 or 쓰로잉)', ha='right', va='top', color='#e74c3c', fontweight='bold')
    ax.text(ax.get_xlim()[1], ax.get_ylim()[0], '평균 이상 + 꾸준함\n(압도적 1황)', ha='right', va='bottom', color='#27ae60', fontweight='bold')
    ax.text(ax.get_xlim()[0], ax.get_ylim()[1], '평균 이하 + 기복있음\n(불안정)', ha='left', va='top', color='#7f8c8d', fontweight='bold')
    ax.text(ax.get_xlim()[0], ax.get_ylim()[0], '평균 이하 + 꾸준함\n(무난함)', ha='left', va='bottom', color='#95a5a6', fontweight='bold')

    ax.set_xlabel('기여도 Ratio (vs Challenger Mean)', fontsize=12)
    ax.set_ylabel('경기력 변동폭 (Std Dev)', fontsize=12)
    ax.set_title(f'[{position}] 챌린저 내부 경쟁력 분석', fontsize=16, fontweight='bold')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.5)

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()
    print(f"   📊 그래프 저장: {save_path}")

def print_top_players(player_stats, position):
    if player_stats.empty: return
    print(f"\n🏆 [{position}] 챌린저 TOP 5 (기여도 순)")
    top5 = player_stats.sort_values(['contribution', 'volatility'], ascending=[False, True]).head(5)
    output = top5[['puuid', 'games', 'contribution', 'volatility', 'win_rate']].copy()
    output['win_rate'] = (output['win_rate']*100).round(1).astype(str) + '%'
    output['contribution'] = output['contribution'].round(3)
    print(output.to_string(index=False))

# ==========================================
# 4. 메인 실행
# ==========================================
def main():
    target_puuids = load_target_puuids(TARGET_PUUID_PATH)
    if not target_puuids: return

    for pos in TARGET_POSITIONS:
        # process_position_data에 target_puuids를 전달하여 내부에서 필터링 수행
        match_stats = process_position_data(pos, target_puuids)
        if match_stats.empty: continue

        player_stats = aggregate_player_final_stats(match_stats, min_games=MIN_GAMES)

        print(f"   ✅ 분석 대상: {len(player_stats)}명")

        pos_dir = BASE_OUTPUT_DIR / pos
        pos_dir.mkdir(parents=True, exist_ok=True)

        player_stats.to_csv(pos_dir / "challenger_calc_stats.csv", index=False)
        print_top_players(player_stats, pos)
        plot_scatter(player_stats, pos, pos_dir / "challenger_calc_scatter.png")

    print(f"\n✅ 분석 완료! 결과 폴더: '{BASE_OUTPUT_DIR}'")

if __name__ == "__main__":
    main()

📁 챌린저 기준 분석 결과 폴더: output/challengers_only_20251207_224827
📋 타겟 플레이어(Challengers) 300명 로드 완료.

📂 [TOP] 데이터 분석 시작...
   ✂️ 데이터 필터링: 2,177,986행 → 526,447행 (챌린저 데이터만 사용)
   AI 예측 수행 중...
   경기별 기여도 집계 중...
   선수별 최종 통계 산출 (최소 30판)...
   ✅ 분석 대상: 90명

🏆 [TOP] 챌린저 TOP 5 (기여도 순)
                                                                         puuid  games  contribution  volatility win_rate
Q8tuLUQmCbypZIlhhmohcibyLQMb8LpkI4Q_z4m52Ivz9u8ZFJPBrUzxnAL_XEt3zgtKgohw_ZV85A     53         1.095    0.139871    56.6%
_2gy_DC46aRGaj02qVS4G6kbPavUSh8Sn3pwDFx-ww2bLq4f-g_5vFY1iFrAsDwciaOJlU4qab90Xg     37         1.068    0.161476    51.4%
abcXImUS5_TGuWRJpVoSroMMU06qORFTXQmSAt5BQV59-bqkCQsK_R7t9e7ACo5WJZO5x3iRCKrGCg    189         1.065    0.146364    53.4%
e95ZeBjm1vACu4gOG_qASC0rKVu88c0ML8h23jjcwW8El4zo9KHlCgN-5nf3B85-AugDVHtMVmKgPw    141         1.059    0.127407    55.3%
NwNAsPf7G8r9Ug6cS7PiSzXm-zkxMel0kgNqPVRXUdRKbDYdG483otEYjmwsiiEjyGCwntFKXeNhPA    180         1.054    0.120339    52.8%
